In [1]:
import numpy as np
import requests
import io
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
def make_request(filename):
    response = requests.get('https://generalzach.s3-us-west-2.amazonaws.com/'+filename)
    response.raise_for_status()
    data = np.load(io.BytesIO(response.content)) 
    return data

def load_training_data():
    print("Making request...")
    training_data = make_request('training_data_256_balanced_sample_new_500.npz')
    print("Success!")
    return training_data['a'],training_data['b']

def load_validation_data():
    print("Making request...")
    validation_data = make_request('validation_data_256_balanced_sample_new_test.npz')
    print("Success!")
    return validation_data['a'],validation_data['b']


In [3]:
training_data,training_labels = load_training_data()
validation_data,validation_labels = load_validation_data()

print("\nLoading training data...")

Making request...
Success!
Making request...
Success!

Loading training data...


In [4]:
training_data_generator = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.05,
        height_shift_range=0.05)
#small batch lol it crashed
BATCH_SIZE=5

In [5]:
training_iterator = training_data_generator.flow(training_data,training_labels,batch_size=BATCH_SIZE)

print("\nLoading validation data...")

#Create validation_data_generator, an ImageDataGenerator that just performs pixel normalization:

validation_data_generator = ImageDataGenerator(rescale=1./255)

#Use validation_data_generator.flow_from_directory(...) to load the validation data from the 'validation_data' folder:

validation_iterator = validation_data_generator.flow(validation_data,validation_labels,batch_size=BATCH_SIZE)



Loading validation data...


In [6]:
#Rebuilds our model from the previous exercise, with convolutional and max pooling layers:
print("\nBuilding model...")
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(256, 256, 1)))
model.add(tf.keras.layers.Conv2D(2, 5, strides=3, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(5, 5), strides=(5,5)))
model.add(tf.keras.layers.Conv2D(4, 3, strides=1, activation="relu")) 
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2,2), strides=(2,2)))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(2,activation="softmax"))

model.summary()



Building model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 84, 84, 2)         52        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 2)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 4)         76        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 4)          0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 196)               0         
                                                                 
 dense (Dense)               (None, 2

In [7]:
#Compile the model with an Adam optimizer, Categorical Cross Entropy Loss, and Accuracy and AUC metrics:

print("\nCompiling model...")

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()],
)


Compiling model...


In [8]:
#Use model.fit(...) to train and validate our model for 2 epochs:

print("\nTraining model...")

model.fit(
        training_iterator,
        steps_per_epoch=len(training_data)/BATCH_SIZE, #training_iterator.samples/BATCH_SIZE,
        epochs=4,
       validation_data=validation_iterator,
        validation_steps=len(validation_data)/BATCH_SIZE) 



Training model...
Epoch 1/4
200/200 [==============================] - 5s 24ms/step - loss: 0.6424 - categorical_accuracy: 0.6480 - auc: 0.7020 - val_loss: 0.5224 - val_categorical_accuracy: 0.7825 - val_auc: 0.8556
Epoch 2/4
200/200 [==============================] - 5s 23ms/step - loss: 0.4884 - categorical_accuracy: 0.7930 - auc: 0.8571 - val_loss: 0.4136 - val_categorical_accuracy: 0.8150 - val_auc: 0.8942
Epoch 3/4
200/200 [==============================] - 5s 24ms/step - loss: 0.4301 - categorical_accuracy: 0.8160 - auc: 0.8897 - val_loss: 0.3790 - val_categorical_accuracy: 0.8325 - val_auc: 0.9104
Epoch 4/4
200/200 [==============================] - 5s 22ms/step - loss: 0.3291 - categorical_accuracy: 0.8630 - auc: 0.9348 - val_loss: 0.3726 - val_categorical_accuracy: 0.8450 - val_auc: 0.9138


In [9]:
#it crashed before I can evaluate :(